In [2]:
import math
import torch
import transformers
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
import evaluate
from datasets import DatasetDict, Dataset
import numpy as np
import random
import os
import wandb

In [12]:
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
model = AutoModelForCausalLM.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token
wandb.init(project="testing-ts-training")


In [3]:
ds = None
train_split = 0.8
validation_split = 0.2
with open("final_lyrics.txt", "r") as f:
    arr = f.readlines()
    train, validation, test = np.split(arr, [int(len(arr)*train_split), int(len(arr)*validation_split)])
    datasets = DatasetDict(
      {
          'train': Dataset.from_dict({'text': list(train)}),
          'validation': Dataset.from_dict({'text': list(validation)}),
          'test': Dataset.from_dict({'text': list(test)})
      }
      
  )
    


In [7]:
def tokenizer_func(data):
    return tokenizer(data["text"], truncation=True)
    
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

block_size = 128

tokenized_datasets = datasets.map(tokenizer_func, batched=True, remove_columns=["text"], num_proc=4)
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    num_proc=4,
    batch_size=1000,
)

# tokenizer.decode(lm_datasets["train"][1]["input_ids"])

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

...

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [66]:
# data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)
# training_args = TrainingArguments(output_dir="ts-testing-training-model",evaluation_strategy="epoch",learning_rate=2e-5,weight_decay=0.01, num_train_epochs=25, report_to='wandb', use_mps_device=True, eval_steps=1, logging_steps=4)
# trainer = Trainer(model=model, args=training_args, train_dataset=lm_datasets['train'], eval_dataset=lm_datasets['test'], data_collator=data_collator)

# trainer.train()

PyTorch: setting up devices
/Users/bhavya/projects/hugging-face-playground/.env/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 292
  Num Epochs = 25
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 925
  Number of trainable parameters = 81912576
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


  0%|          | 0/925 [00:00<?, ?it/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 3.3932, 'learning_rate': 1.9913513513513515e-05, 'epoch': 0.11}
{'loss': 3.29, 'learning_rate': 1.9827027027027028e-05, 'epoch': 0.22}
{'loss': 3.2532, 'learning_rate': 1.974054054054054e-05, 'epoch': 0.32}
{'loss': 3.3229, 'learning_rate': 1.9654054054054054e-05, 'epoch': 0.43}
{'loss': 3.1418, 'learning_rate': 1.956756756756757e-05, 'epoch': 0.54}
{'loss': 3.2947, 'learning_rate': 1.9481081081081084e-05, 'epoch': 0.65}
{'loss': 3.3074, 'learning_rate': 1.9394594594594597e-05, 'epoch': 0.76}
{'loss': 3.2243, 'learning_rate': 1.930810810810811e-05, 'epoch': 0.86}
{'loss': 3.0769, 'learning_rate': 1.9221621621621623e-05, 'epoch': 0.97}


***** Running Evaluation *****
  Num examples = 295
  Batch size = 8


...

{'eval_loss': 2.9911916255950928, 'eval_runtime': 6.0338, 'eval_samples_per_second': 48.891, 'eval_steps_per_second': 6.132, 'epoch': 1.0}
{'loss': 3.1872, 'learning_rate': 1.9135135135135137e-05, 'epoch': 1.08}
{'loss': 3.0565, 'learning_rate': 1.904864864864865e-05, 'epoch': 1.19}
{'loss': 3.2667, 'learning_rate': 1.8962162162162163e-05, 'epoch': 1.3}
{'loss': 3.0106, 'learning_rate': 1.8875675675675676e-05, 'epoch': 1.41}
{'loss': 2.9659, 'learning_rate': 1.878918918918919e-05, 'epoch': 1.51}
{'loss': 3.0265, 'learning_rate': 1.8702702702702706e-05, 'epoch': 1.62}
{'loss': 3.0242, 'learning_rate': 1.861621621621622e-05, 'epoch': 1.73}
{'loss': 3.0658, 'learning_rate': 1.8529729729729732e-05, 'epoch': 1.84}
{'loss': 2.9924, 'learning_rate': 1.8443243243243245e-05, 'epoch': 1.95}


***** Running Evaluation *****
  Num examples = 295
  Batch size = 8


  0%|          | 0/37 [00:00<?, ?it/s]

{'eval_loss': 2.887949228286743, 'eval_runtime': 6.0392, 'eval_samples_per_second': 48.847, 'eval_steps_per_second': 6.127, 'epoch': 2.0}
{'loss': 3.0231, 'learning_rate': 1.8356756756756758e-05, 'epoch': 2.05}
{'loss': 2.9795, 'learning_rate': 1.827027027027027e-05, 'epoch': 2.16}
{'loss': 2.9884, 'learning_rate': 1.8183783783783785e-05, 'epoch': 2.27}
{'loss': 3.0773, 'learning_rate': 1.8097297297297298e-05, 'epoch': 2.38}
{'loss': 2.9212, 'learning_rate': 1.8010810810810814e-05, 'epoch': 2.49}
{'loss': 2.9708, 'learning_rate': 1.7924324324324324e-05, 'epoch': 2.59}


NotImplementedError: The operator 'aten::histc' is not currently implemented for the MPS device. If you want this op to be added in priority during the prototype phase of this feature, please comment on https://github.com/pytorch/pytorch/issues/77764. As a temporary fix, you can set the environment variable `PYTORCH_ENABLE_MPS_FALLBACK=1` to use the CPU as a fallback for this op. WARNING: this will be slower than running natively on MPS.

In [ ]:
eval_results = trainer.evaluate()

print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [5]:
model = AutoModelForCausalLM.from_pretrained("output/ts-model")
trainer = Trainer(model=model)

trainer.push_to_hub()
# tokenizer = AutoModelForCausalLM.from_pretrained("output/ts-model")



loading configuration file output/ts-model/config.json
Model config GPT2Config {
  "_name_or_path": "output/ts-model",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 6,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype"

Upload file pytorch_model.bin:   0%|          | 32.0k/318M [00:00<?, ?B/s]

Upload file training_args.bin: 100%|##########| 3.37k/3.37k [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/BhavyaMuni/tmp_trainer
   ca1e807..4cd21b1  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Causal Language Modeling', 'type': 'text-generation'}}
To https://huggingface.co/BhavyaMuni/tmp_trainer
   4cd21b1..ecb3418  main -> main



'https://huggingface.co/BhavyaMuni/tmp_trainer/commit/4cd21b1f211e2b71d73b3779e1a6dc174b254b3f'